# Init

In [ ]:
import cv2 as cv
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.ndimage
import skimage.filters  # for intellisense
import skimage.util  # for intellisense

In [ ]:
sns.set(style='dark')
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['image.cmap'] = 'viridis'
np.set_printoptions(threshold=20, edgeitems=10, linewidth=140, precision=3, suppress=True)

In [ ]:
rgb = cv.imread('./data/robot.bmp')[..., ::-1]
rgb = skimage.util.img_as_float32(rgb)
rgb.shape, rgb.dtype, rgb.min(), rgb.max()

In [ ]:
gray = cv.cvtColor(rgb, cv.COLOR_RGB2GRAY)
gray = skimage.util.img_as_float32(gray)
gray.shape, gray.dtype, gray.min(), gray.max()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained')
axes[0].imshow(rgb)
axes[1].imshow(gray, cmap='gray', vmin=0, vmax=1);

# Spatial processing

# Additive noise model

We assume that the noise value $\nu$ at each pixel is an independent random variable with zero and standard deviation $\sigma$ and that the observed value of image $I(x,y)$ at position $(x,y)$ is a sum of the true (but unknown) value $\bar{I}(x,y)$ and noise $\nu$
$$
I(x,y) = \bar{I}(x,y) + \nu
$$
Usually, $\nu$ comes from a normal (Gaussian) distribution, i.e. $\nu\sim\mathcal{N}(0,\sigma)$.

In [ ]:
rgb_gn = skimage.util.random_noise(rgb, mode='gaussian', var=0.01)
rgb_gn.shape, rgb_gn.dtype, rgb_gn.min(), rgb_gn.max()

In [ ]:
gray_gn = skimage.util.random_noise(gray, mode='gaussian', var=0.01)
gray_gn.shape, gray_gn.dtype, gray_gn.min(), gray_gn.max()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained')
axes[0].imshow(rgb_gn)
axes[1].imshow(gray_gn, cmap='gray', vmin=0., vmax=1.);

# Noise filtering using multiple image averaging

If we are able to capture the same scene $N$ times as images $I_n$, we may estimate the true value of the image as
$$
\hat{I}(x,y) = \frac{1}{N}\cdot\sum_{n=1}^{N}{I_n(x,y)} + \frac{1}{N}\cdot\sum_{n=1}^{N}{\nu_{n,x,y}}
$$
The second term describes the noise, which, under the assumption that the expected value of $\nu$ is $\text{E}(\nu) = 0$ and $\text{var}(\nu) = \sigma^2$, is again a random value with zero mean and a standard deviation $\sigma / N$. As $N \rightarrow \infty$, $\sigma / N \rightarrow 0$.

Repeated capture (burst) of the same scene and then averaging is what basically happens in many mobile cameras to reduce noise especially under insufficient lightning conditions.

In [ ]:
rgb_gns = np.array([
    skimage.util.random_noise(rgb, mode='gaussian', var=0.01)
    for i in range(10)
])
rgb_gns.dtype, rgb_gns.shape, rgb_gns.min(), rgb_gns.max()

In [ ]:
rgb_gns_r = rgb_gns.mean(axis=0)
rgb_gns_r.shape, rgb_gns_r.shape, rgb_gns_r.min(), rgb_gns_r.max()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained')
axes[0].imshow(rgb_gn)
axes[1].imshow(rgb_gns_r);

# Noise filtering using local averaging (image smoothing)

More often than not, we are stuck with only one image and avering is then performed in a local neighborhood. Assuming a square region of size $K \times K$:
$$
\hat{I}(x,y) = \frac{1}{K^2}\cdot\sum_{i=-K/2}^{+K/2}\sum_{j=-K/2}^{+K/2}{I(x+i,y+j)}
$$
Results are acceptable if the noise is in smaller size than the smallest objects of interest in the image, but **blurring of edges** is a serious disadvantage.

In [ ]:
def local_average(img: np.ndarray, wnd_size: int = 3) -> np.ndarray:
    img_r = np.zeros_like(img)
    half = wnd_size // 2
    for row in range(half, img.shape[0] - half):
        for col in range(half, img.shape[1] - half):
            window = img[row-half:row+half+1, col-half:col+half+1]
            img_r[row, col] = window.mean(axis=(0, 1))
    return img_r

In [ ]:
gray_gn_r = local_average(gray_gn, wnd_size=3)
gray_gn_r.shape, gray_gn_r.shape, gray_gn_r.min(), gray_gn_r.max()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained')
axes[0].imshow(gray_gn, cmap='gray', vmin=0, vmax=1)
axes[1].imshow(gray_gn_r, cmap='gray', vmin=0, vmax=1);

In [ ]:
rgb_gn_r = local_average(rgb_gn, wnd_size=3)
rgb_gn_r.shape, rgb_gn_r.shape, rgb_gn_r.min(), rgb_gn_r.max()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained')
axes[0].imshow(rgb_gn)
axes[1].imshow(rgb_gn_r);

# Convolution

Discrete 2D convolution is
$$
I'(x,y) = \sum_{i=-\infty}^{+\infty}\sum_{j=-\infty}^{+\infty}{I(x-i,y-j) \cdot h(i,j)}
$$
or
$$
I' = I * h
$$

Illustration of convolution:

<img src="figures/spatial_filtering-convolution_animation.gif" alt="Convolution" style="height: 4in;"/>

**Convolution properties**

Commutativity
$$
I * h = h * I
$$

Linearity
$$
(s \cdot I) * h = I * (s \cdot h) = s \cdot (I * h) \\
(I_1 + I_2) * h = (I_1 * h) + (I_2 * h)
$$

Associativity
$$
(I * h_1) * h_2 = I * (h_1 * h_2)
$$

In [ ]:
def conv2d(gray: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    out = np.zeros_like(gray)
    img_h, img_w = gray.shape
    k_h, k_w = kernel.shape
    for y in range(k_h // 2, img_h - k_h // 2):
        for x in range(k_w // 2, img_w - k_w // 2):
            window = gray[y-k_h//2:y+k_h//2+1, x-k_w//2:x+k_w//2+1]
            out[y, x] = np.sum(kernel * window)
    return out

In [ ]:
kernel = 1/9 * np.ones((3, 3))
kernel

In [ ]:
gray_gn_rc = conv2d(gray_gn, kernel)
gray_gn_rc.shape, gray_gn_rc.shape, gray_gn_rc.min(), gray_gn_rc.max()

##

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained')
axes[0].imshow(gray_gn_r, cmap='gray', vmin=0, vmax=1)
axes[1].imshow(gray_gn_rc, cmap='gray', vmin=0, vmax=1);

In [ ]:
np.sum(np.abs(gray_gn_rc - gray_gn_r))

## Choice of kernel: blur, edges, emboss, identity, outline, sharpen

In [ ]:
kernels = {
    'identity': np.array([
        [0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.],
    ]),
    'local averaging': np.array([
        [1/9, 1/9, 1/9],
        [1/9, 1/9, 1/9],
        [1/9, 1/9, 1/9],
    ]),
    'Gaussian-like averaging (blurring)': np.array([
        [1/16, 1/8, 1/16],
        [1/8,  1/4, 1/8],
        [1/16, 1/8, 1/16],
    ]),
    'sharpen': np.array([
        [ 0., -1.,  0.],
        [-1.,  5., -1.],
        [ 0., -1.,  0.],
    ]),
    'emboss': np.array([
        [-2., -1., 0.],
        [-1.,  1., 1.],
        [0.,   1., 2.],
    ]),
    'outline': np.array([
        [-1., -1., -1.],
        [-1.,  8., -1.],
        [-1., -1., -1.],
    ]),
    'horizontal Sobel edges': np.array([
        [-1., -2., -1.],
        [0.,  0.,  0. ],
        [1.,  2.,  1. ],
    ]),
    'vertical Sobel edges': np.array([
        [-1., 0., 1.],
        [-2., 0., 2.],
        [-1., 0., 1.],
    ]),
}

In [ ]:
gray_convs = {
    name: conv2d(gray, kernel)
    for name, kernel in kernels.items()
}

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, layout='constrained', figsize=tuple(2*np.array(plt.rcParams['figure.figsize'])))
for i, (name, gray_conv) in enumerate(gray_convs.items()):
    axes.flat[i].imshow(gray_conv, cmap='gray', vmin=0, vmax=1)
    axes.flat[i].set_title(name)

## Implementations of convolution: scipy

In [ ]:
%%time

gray_gn_rscp = scipy.signal.convolve2d(
    gray_gn,
    kernels['local averaging'],
    mode = 'full',
    boundary = 'symm'
#     fillvalue=0
)
gray_gn_rscp.shape, gray_gn_rscp.dtype, gray_gn_rscp.min(), gray_gn_rscp.max()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, layout='constrained')
axes[0].imshow(gray_gn_rc, cmap='gray', vmin=0, vmax=1)  # using conv2d
axes[1].imshow(gray_gn_rscp, cmap='gray', vmin=0, vmax=1);  # using scipy.signal.convolve

### RGB: filter each channel independently

In [ ]:
%%time

rgb_gn_rscp = np.zeros_like(rgb_gn)
for c in range(3):
    rgb_gn_rscp[:, :, c] = scipy.signal.convolve2d(
        rgb_gn[:, :, c],
        kernel,
        boundary='symm',
        mode='same'
    )

In [ ]:
plt.imshow(rgb_gn_rscp, cmap='gray', vmin=0, vmax=1);

In [ ]:
# THIS DOES SOMETHING DIFFERENT!!!
tmp_ = scipy.signal.convolve(
    rgb_gn,
    np.stack((kernel, kernel, kernel)) / 3,
#     boundary='symm',  # it doesn't support this argument
    mode='same',
)
tmp_.shape

In [ ]:
plt.imshow(tmp_, cmap='gray', vmin=0, vmax=1);

## Implementations of convolutions: OpenCV

In [ ]:
gray_gn_rcv = cv.filter2D(gray_gn, -1, kernel)

In [ ]:
plt.imshow(gray_gn_rcv, cmap='gray', vmin=0, vmax=1);

In [ ]:
rgb_gn_rcv = cv.filter2D(rgb_gn, -1, kernel)

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(rgb_gn_rcv);
axes[1].imshow(gray_gn_rcv, cmap='gray', vmin=0, vmax=1);

In [ ]:
plt.imshow(
    scipy.signal.convolve2d(gray_gn_rcv, kernel, boundary='symm', mode='same')
    -
    cv.filter2D(gray_gn_rcv, -1, kernel, borderType=cv.BORDER_REFLECT)
)
plt.colorbar();

# Separable filters

In [ ]:
kernel_1d = scipy.signal.windows.gaussian(11, 2.)
kernel_1d

In [ ]:
kernel_1d /= kernel_1d.sum()
kernel_1d

In [ ]:
plt.plot(kernel_1d);
plt.grid();

In [ ]:
kernel_1d[:, None].shape, kernel_1d[None, :].shape

In [ ]:
kernel_2d = np.dot(kernel_1d[:, None], kernel_1d[None, :])
kernel_2d.shape, kernel_2d.sum()

In [ ]:
np.set_printoptions(precision=2, floatmode='fixed')
kernel_2d

In [ ]:
plt.imshow(kernel_2d)
plt.colorbar();

In [ ]:
gray_gn_s1x2d = scipy.ndimage.convolve(gray_gn, kernel_2d)

In [ ]:
plt.imshow(gray_gn_s1x2d, cmap='gray', vmin=0, vmax=1);

In [ ]:
def conv2d_sep(img: np.ndarray, kernel1: np.ndarray, kernel2: np.ndarray) -> np.ndarray:
    out = scipy.ndimage.convolve(img, kernel1)
    out = scipy.ndimage.convolve(out, kernel2)
    return out

In [ ]:
gray_gn_s2x1d = conv2d_sep(gray_gn, kernel_1d[None, :], kernel_1d[:, None])  # convolve rows then columns

In [ ]:
fig.get_size_inches()[1], axes[2].get_window_extent().transformed(fig.dpi_scale_trans.inverted()).height

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=plt.figaspect(0.5))
axes[0].imshow(gray_gn_s1x2d, cmap='gray', vmin=0, vmax=1);
axes[1].imshow(gray_gn_s2x1d, cmap='gray', vmin=0, vmax=1);
im2 = axes[2].imshow(np.abs(gray_gn_s2x1d - gray_gn_s1x2d));
fig.colorbar(im2, ax=axes[2], shrink=axes[2].get_window_extent().transformed(fig.dpi_scale_trans.inverted()).height/fig.get_size_inches()[1]);

In [ ]:
%timeit scipy.ndimage.convolve(gray_gn, kernel_2d)

In [ ]:
%timeit conv2d_sep(gray_gn, kernel_1d[None, :], kernel_1d[:, None])

In [ ]:
def conv2d_sep_faster(img: np.ndarray, kernel1: np.ndarray, kernel2: np.ndarray) -> np.ndarray:
    # scipy.ndimage.convolve1d is faster than scipy.ndimage.convolve
    out = scipy.ndimage.convolve1d(img.T, kernel1)
    out = scipy.ndimage.convolve1d(out.T, kernel2)
    return out

In [ ]:
gray_gn_s2x1d_ = conv2d_sep_faster(gray_gn, kernel_1d, kernel_1d)

In [ ]:
%timeit conv2d_sep_faster(gray_gn, kernel_1d, kernel_1d)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=plt.figaspect(0.5))
axes[0].imshow(gray_gn_s2x1d, cmap='gray', vmin=0, vmax=1);
axes[1].imshow(gray_gn_s2x1d_, cmap='gray', vmin=0, vmax=1);
im2 = axes[2].imshow(np.abs(gray_gn_s2x1d - gray_gn_s2x1d_));
fig.colorbar(im2, ax=axes[2], shrink=axes[2].get_window_extent().transformed(fig.dpi_scale_trans.inverted()).height/fig.get_size_inches()[1]);

# Salt & pepper noise: non-linear median filter

In [ ]:
gray_sp = skimage.util.random_noise(gray, mode='s&p', amount=0.05)
rgb_sp = skimage.util.random_noise(rgb, mode='s&p', amount=0.05)

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(rgb_sp);
axes[1].imshow(gray_sp, cmap='gray', vmin=0, vmax=1);

In [ ]:
gray_sp_ravg = scipy.ndimage.convolve(gray_sp, 1/9*np.ones((3, 3)))

In [ ]:
plt.imshow(gray_sp_ravg, cmap='gray', vmin=0, vmax=1);

## Local median of 3x3 neighborhood

In [ ]:
def median_filter(gray: np.ndarray, wnd_size: int) -> np.ndarray:
    gray_sp_rmed = np.zeros_like(gray_sp)
    half = wnd_size // 2
    for row in range(half, gray_sp.shape[0] - half):
        for col in range(half, gray_sp.shape[1] - half):
            window = gray_sp[row-half:row+half+1, col-half:col+half+1]
            gray_sp_rmed[row, col] = np.median(window)
    return gray_sp_rmed

In [ ]:
gray_sp_rmed = median_filter(gray_sp, 3)
gray_sp_rmed.shape

In [ ]:
plt.imshow(gray_sp_rmed, cmap='gray', vmin=0, vmax=1);

### OpenCV

In [ ]:
gray_sp_rmed_cv = cv.medianBlur(gray_sp, 3)

In [ ]:
plt.imshow(gray_sp_rmed_cv, cmap='gray', vmin=0, vmax=1);

### Scipy: arbitrarily shaped neighborhoood (`selem`)

In [ ]:
gray_sp_rmed_scp = scipy.ndimage.median_filter(gray_sp, 3)

In [ ]:
plt.imshow(gray_sp_rmed_scp, cmap='gray', vmin=0, vmax=1);

In [ ]:
# Let's use cross-shaped neighborhood
selem = np.zeros((3, 3))
selem[1, :] = 1
selem[:, 1] = 1
selem

In [ ]:
gray_sp_rmedc_scp = skimage.filters.median(gray_sp, footprint=selem)

In [ ]:
plt.imshow(gray_sp_rmedc_scp, cmap='gray', vmin=0, vmax=1);

In [ ]:
# note the dtype change
gray_sp_rmed.dtype, gray_sp_rmed.shape, gray_sp_rmed.min(), gray_sp_rmed.max()

### scikit-image: arbitrarily shaped neighborhoood (`footprint`)

In [ ]:
gray_sp_rmedc_ski = skimage.filters.median(gray_sp, footprint=selem)

In [ ]:
plt.imshow(gray_sp_rmedc_ski, cmap='gray', vmin=0, vmax=1);

# Non-linear filtering: Bilateral filter

$$
\tilde{I}(\bold{x}) = \frac{1}{W_p} \sum_{\bold{u} \in \Omega}{ f(\|I(\bold{u}) - I(\bold{x})\|) \cdot g(\|\bold{u}-\bold{v}\|) \cdot I(x,y)},
$$
where
- $\bold{x} = [x,y]^\top$
- $\bold{u} = [u,v]^\top$
- $f$ is a function that weighs how different intensities at positions $(x,y)$ and $(u,v)$ are - $f$ is usually a Gaussian function with $\sigma_\textrm{color}$,
- $g$ is a function that weighs how far apart positions $(x,y)$ and $(u,v)$ are - again, $g$ is usually a Gaussian function with $\sigma_\textrm{spatial}$,
- $\Omega$ is a local neigborhood around which the smoothing using $f$ and $g$ for pixel $(x,y)$ is performed,
- normalization factor $W_p$ ensures pixel weights sum to 1.0:
$$
W_p = \sum_{\bold{u} \in \Omega}{ f(\|I(\bold{u}) - I(\bold{x})\|) \cdot g(\|\bold{u}-\bold{v}\|) }
$$

In [ ]:
def denoise_bilateral_interact(img, cax, sigma_color: float = 0.05, sigma_spatial: float = 2):
    img_r = skimage.restoration.denoise_bilateral(
        img,
        sigma_color = sigma_color,
        sigma_spatial = sigma_spatial,
        channel_axis = cax
    )
    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(img, cmap='gray', vmin=0., vmax=1.);
    axes[1].imshow(img_r, cmap='gray', vmin=0., vmax=1.);

ipywidgets.interact(
    denoise_bilateral_interact,
    img = ipywidgets.fixed(rgb_gn),
    cax = ipywidgets.fixed(2),
    sigma_color = (0., 0.3, 0.01),
    sigma_spatial = (0., 30.),
    continuous_update=False
);

In [ ]:
# salt & pepper noise
ipywidgets.interact(
    denoise_bilateral_interact,
    img = ipywidgets.fixed(rgb_sp),
    cax = ipywidgets.fixed(2),
    sigma_color = (0., 0.3, 0.01),
    sigma_spatial = (0., 30.),
    continuous_update = False
);

## OpenCV

In [ ]:
def denoise_bilateral_interact_cv(img, sigma_color: float = 128., sigma_spatial: float = 2.):
    img_r = cv.bilateralFilter(
        img,
        None,
        sigma_color,
        sigma_spatial
    )
    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(img, cmap='gray', vmin=0., vmax=255);
    axes[1].imshow(img_r, cmap='gray', vmin=0., vmax=255);

ipywidgets.interact(
    denoise_bilateral_interact_cv,
    img = ipywidgets.fixed(skimage.util.img_as_ubyte(rgb_sp)),
    sigma_color = (0., 255.),
    sigma_spatial = (0., 30.),
    continuous_update = True
);

# Non-linear filtering: Non-local means

Example from https://scikit-image.org/docs/dev/auto_examples/filters/plot_nonlocal_means.html#sphx-glr-auto-examples-filters-plot-nonlocal-means-py

In [ ]:
# Algorithm needs to know approximate value of the standard deviation of the noise
sigma_est = np.mean(skimage.restoration.estimate_sigma(rgb_gn, channel_axis=2))
sigma_est

In [ ]:
# slow algorithm
rgb_gn_rnlm1 = skimage.restoration.denoise_nl_means(
    rgb_gn,
    h = 1.15 * sigma_est,  # threshold for comparing patch similarity
    fast_mode = False,
    patch_size = 5,
    patch_distance = 6,  # how far to search for similar patches
    channel_axis = 2
)

In [ ]:
# slow algorithm, sigma provided
rgb_gn_rnlm2 = skimage.restoration.denoise_nl_means(
    rgb_gn,
    h = 0.8 * sigma_est,
    sigma = sigma_est,
    fast_mode = False,
    patch_size = 5,
    patch_distance = 6,
    channel_axis = 2
)

In [ ]:
# fast algorithm
rgb_gn_rnlm3 = skimage.restoration.denoise_nl_means(
    rgb_gn,
    h = 0.8 * sigma_est,
    fast_mode = True,
    patch_size = 5,
    patch_distance = 6,
    channel_axis = 2
)

In [ ]:
# fast algorithm, sigma provided
rgb_gn_rnlm4 = skimage.restoration.denoise_nl_means(
    rgb_gn,
    h = 0.6 * sigma_est,
    sigma = sigma_est,
    fast_mode = True,
    patch_size = 5,
    patch_distance = 6,
    channel_axis = 2
)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 16))
axes[0, 0].imshow(rgb);
axes[0, 1].imshow(rgb_gn_rnlm1);
axes[0, 2].imshow(rgb_gn_rnlm2);
axes[1, 0].imshow(rgb_gn);
axes[1, 1].imshow(rgb_gn_rnlm3);
axes[1, 2].imshow(rgb_gn_rnlm4);